Perplexity

In [61]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [63]:
pip install transformers==4.28.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [64]:
import pickle

In [82]:
#Load the model from the file
with open('EminemBetamodel', 'rb') as file:
    model = pickle.load(file) 

In [75]:
import random
import pronouncing
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
def extract_rhymes(text):
    lines = text.split('\n')
    rhyming_pairs = []

    for i in range(len(lines) - 1):
        line1 = lines[i].split()
        line2 = lines[i + 1].split()

        if line1 and line2:
            last_word1 = line1[-1]
            last_word2 = line2[-1]
            if last_word1 in pronouncing.rhymes(last_word2):
                rhyming_pairs.append(lines[i])
                rhyming_pairs.append(lines[i + 1])

    return '\n'.join(rhyming_pairs)

# Prepare the training data
train_file = 'Eminem.txt'
with open(train_file, 'r') as f:
    text = f.read()

rhyming_text = extract_rhymes(text)

with open('rhyming_emiem.txt', 'w') as f:
    f.write(rhyming_text)

class GPT2RhymeModel(GPT2LMHeadModel):
    def __init__(self, config, tokenizer):
        super().__init__(config)
        self.tokenizer = tokenizer
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        outputs = super().forward(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            labels=labels,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        lm_logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss()

        # Compute standard language modeling loss
        lm_loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))

        # Add custom rhyme loss
        rhyme_loss = compute_rhyme_loss(lm_logits, labels, self.tokenizer)
        total_loss = lm_loss + rhyme_loss.item()
        return (total_loss, outputs)

def compute_rhyme_loss(lm_logits, labels, tokenizer):
    # Convert logits to word tokens
    pred_ids = torch.argmax(lm_logits, dim=-1)
    pred_tokens = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    target_tokens = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rhyme_loss = 0.0
    rhyme_count = 0

    for batch_idx in range(len(pred_tokens)):
        pred_word = pred_tokens[batch_idx]
        target_word = target_tokens[batch_idx]

        if target_word in pronouncing.rhymes(pred_word):
            rhyme_count += 1

    # Calculate the rhyme percentage and convert it to a PyTorch tensor
    rhyme_percentage = 1.0 - (rhyme_count / len(pred_tokens))
    rhyme_loss = torch.tensor(rhyme_percentage, dtype=torch.float32)

    return rhyme_loss
    

In [69]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [83]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import torch.nn as nn

# Load pre-trained model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = "The quick brown fox jumps over the lazy dog."
tokens_tensor = tokenizer.encode(text, return_tensors='pt')

# Calculate the loss
model.eval()
with torch.no_grad():
    outputs = model(tokens_tensor, labels=tokens_tensor)
    loss = outputs.loss

# Calculate the perplexity
perplexity = torch.exp(loss)
print(f'Perplexity: {perplexity.item()}')


Perplexity: 815.455078125


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model2 = GPT2LMHeadModel.from_pretrained('gpt2')

# Encode a text inputs
text = "The quick brown fox jumps over the lazy dog."
tokens_tensor = tokenizer.encode(text, return_tensors='pt')

# Calculate the loss
model.eval()
with torch.no_grad():
    outputs = model2(tokens_tensor, labels=tokens_tensor)
    loss = outputs.loss

# Calculate the perplexity
perplexity = torch.exp(loss)
print(f'Perplexity: {perplexity.item()}')


Perplexity: 162.47128295898438


Rhyme Density

In [2]:
import string
def remove_punctuation(s):
    return ''.join(c for c in s if c not in string.punctuation) 

In [3]:
pip install pronouncing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 18.0 MB/s eta 0:00:00
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6237 sha256=7485de9e44b135c8d9baeb6076290fbedd851a4382eaac2dc91865014cbb5e7e
  Stored in directory: /root/.cache/pip/wheels/05/f6/1d/599c67da1fa48c086d8c49e8fc6bd5f05bc9fa66fb04bed5db
Successfully built pronouncing


In [60]:
import pronouncing
import re

def rhyme(text):
  count=[0]*len(text)
  text=text.split()
  for i in range(len(text)):
    text[i]=remove_punctuation(text[i])
  for k in range(len(text)):
    rhymes = pronouncing.rhymes(text[k])
    for j in range(len(text)):
      if text[j] in rhymes:
        count[k]=count[k]+1
  num=0
  for l in range(len(count)):
    if count[l]>0:
      num=num+1
  pro=num/len(text)  
  print(pro)
rhyme("""And use my enemy's words as strength
To try and draw from, and get inspired off 'em
‘Cause all my life I was told and taught I am not shit
By you wack fuckin' giant sacks of lyin' dog shit
Now you shut up, bitch, I am talkin'
Thought I was full of horseshit
And now you fuckin' worship the ground on which I am walkin'
Me against the world, so what? I'm Brian Dawkins
Versus the whole 0-16 Lions offense
So bring on the Giants, Falcons, and Miami Dolphins
It's the body bag game, bitch, I'm supplyin' coffins""")

0.23232323232323232
